In [1]:
!pip install lightgbm

In [2]:
import pickle
from typing import Any, Dict, Tuple

import pandas as pd
import numpy as np

from tqdm.auto import tqdm

from rectools.models import LightFMWrapperModel
from lightfm import LightFM

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

from lightgbm import LGBMRanker
from rectools import Columns
from lightfm.data import Dataset
from rectools.dataset import Interactions
from rectools.model_selection import TimeRangeSplitter
from rectools.metrics import MAP, NDCG, Precision, Recall, MeanInvUserFreq, Serendipity, calc_metrics
from rectools.tools.ann import UserToItemAnnRecommender

RANDOM_STATE = 1337

Загрузка данных

In [3]:
DATA_DIR_PATH = '/mnt/88fdd009-dda3-49d8-9888-cfd9d9d5910a/ITMO/RecomendationsService/DATA'

interactions = pd.read_csv(f'{DATA_DIR_PATH}/interactions.csv')

# interactions = pd.read_csv('data_original/interactions.csv')
users = pd.read_csv(f'{DATA_DIR_PATH}/users.csv')
items = pd.read_csv(f'{DATA_DIR_PATH}/items.csv')

In [4]:
# Меняем названия колонок для использования rectools
interactions.rename(
    columns={
        'last_watch_dt': Columns.Datetime,
        'watched_pct': Columns.Weight,
    },
    inplace=True,
)
# Меняем тип данных
interactions['datetime'] = interactions['datetime'].astype(np.datetime64)

# Заполняем пропуски
interactions_default_values: Dict[str, Any] = {
   Columns.Datetime: interactions[Columns.Datetime].median(),
    Columns.Weight: 0.,
    'total_dur': 0,
}
interactions.fillna(interactions_default_values, inplace=True)

# Смотрим что получилось
interactions.head(10)

/tmp/ipykernel_7275/1052631975.py:18: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  interactions.fillna(interactions_default_values, inplace=True)


,user_id,item_id,datetime,total_dur,weight
0,176549,9506,2021-05-11,4250,72.0
1,699317,1659,2021-05-29,8317,100.0
2,656683,7107,2021-05-09,10,0.0
3,864613,7638,2021-07-05,14483,100.0
4,964868,9506,2021-04-30,6725,100.0
5,1032142,6686,2021-05-13,11286,100.0
6,1016458,354,2021-08-14,1672,25.0
7,884009,693,2021-08-04,703,14.0
8,648682,1449,2021-06-13,26246,75.0
9,203219,13582,2021-08-22,6975,100.0


In [5]:
max_date = interactions[Columns.Datetime].max()
min_date = interactions[Columns.Datetime].min()

print(f'min дата в interactions: {min_date}')
print(f'max дата в interactions: {max_date}')
print(f'Продолжительность: {max_date - min_date}')

min дата в interactions: 2021-03-13 00:00:00
max дата в interactions: 2021-08-22 00:00:00
Продолжительность: 162 days 00:00:00


Разобьем данные на трейн, тест и валидацию

In [6]:
ranker_days_count = 30

ranker_data = interactions[
    (interactions[Columns.Datetime] >= max_date - pd.Timedelta(days=7))
]

train_size = 0.7
val_size = 0.15
test_size = 0.15


train_val_users, test_users = train_test_split(
    ranker_data['user_id'].unique(), random_state=RANDOM_STATE, test_size=test_size
)

train_users, val_users = train_test_split(
    train_val_users, random_state=RANDOM_STATE, test_size=val_size / (train_size + val_size)  # 15% от общего размера
)

In [7]:
def encode_cat_cols(df: pd.DataFrame, cat_cols) -> Tuple[pd.DataFrame, Dict]:
    cat_col_encoding = {}  # словарь с категориями

    for col in cat_cols:
        cat_col = df[col].astype('category').cat
        cat_col_encoding[col] = cat_col.categories
        df[col] = cat_col.codes.astype('category')
    return df, cat_col_encoding

users_cat_cols = [
    # 'user_id',
     'age', 'income', 'sex', 'kids_flg'
]
users, users_cat_col_encoding = encode_cat_cols(users, users_cat_cols)

# None уже кодируется как -1
users_cat_col_encoding['income'], users['income'].unique()

(Index(['income_0_20', 'income_150_inf', 'income_20_40', 'income_40_60',
        'income_60_90', 'income_90_150'],
       dtype='object'),
 [4, 2, 3, 0, -1, 5, 1]
 Categories (7, int64): [-1, 0, 1, 2, 3, 4, 5])

In [8]:
base_models_data = interactions[
    (interactions[Columns.Datetime] < max_date - pd.Timedelta(days=ranker_days_count))
]

In [9]:
base_users = base_models_data[Columns.User].unique()
base_items = base_models_data[Columns.Item].unique()

In [10]:
lightfm_dataset = Dataset()
lightfm_user_ids = base_models_data['user_id'].unique()
lightfm_item_ids = base_models_data['item_id'].unique()
lightfm_dataset.fit(lightfm_user_ids, lightfm_item_ids)

In [ ]:
# так как обучаем модель первого уровнять, то пока без фичей
dataset = Dataset.construct(
    interactions_df=base_models_data,
    user_features_df=None,
    item_features_df=None,
)

In [13]:
interactions_matrix, weights_matrix = lightfm_dataset.build_interactions(
    zip(*base_models_data[['user_id', 'item_id', Columns.Weight]].values.T)
)
weights_matrix = weights_matrix.tocsr()

Теперь обучим модель первого уровня

In [14]:
K_RECOS = 10
RANDOM_STATE = 1337
NUM_THREADS = 16
N_FACTORS = 32
N_EPOCHS = 20
USER_ALPHA = 0
ITEM_ALPHA = 0
LEARNING_RATE = 0.05

In [15]:
# обучим модель и сохраним её
inner_model = LightFM(
    no_components=N_FACTORS,
    loss='warp',
    random_state=RANDOM_STATE,
    learning_rate=0.055
)

lightFM_model = LightFMWrapperModel(
    inner_model,
    epochs=N_EPOCHS,
    num_threads=NUM_THREADS,
)

lightFM_model.fit(dataset)

with open("lightFM_model.pkl", "wb") as file:
    pickle.dump(lightFM_model, file)

In [17]:
# генерируем кандидатов для модели второго уровня
candidates = lightFM_model.recommend(users=base_users, dataset=dataset, k=100, filter_viewed=False)
candidates.to_csv('candidates.csv', index=False)

In [3]:
candidates.head(10)

,user_id,item_id,score,rank
0,176549,13018,3.455198,1
1,176549,5693,3.211725,2
2,176549,7571,3.162938,3
3,176549,9728,3.113396,4
4,176549,1785,3.091236,5
5,176549,16166,3.073796,6
6,176549,11310,3.025889,7
7,176549,7626,3.023241,8
8,176549,14317,3.005028,9
9,176549,101,2.958446,10


Раcчёт метрик

In [15]:
# Считаем метрики
def calc_metrics_(candidates_df, rank_col: str) -> Dict[str, float]:
    metrics = {
        'ndcg@10': NDCG(k = 10),
        'map@10': MAP(k = 10),
        'Precision@10': Precision(k = 10),
        'recall@10': Recall(k = 10),
        'novelty@10': MeanInvUserFreq(k = 10),
    }
    return calc_metrics(
        metrics=metrics,
        reco=(
            candidates_df
            .rename(columns={rank_col: Columns.Rank})
            [[Columns.User, Columns.Item, Columns.Rank]]
            [candidates_df[Columns.User].isin(test_users)]
        ),
        interactions=(
            ranker_data
            [[Columns.User, Columns.Item, Columns.Datetime, Columns.Weight]]
            [ranker_data[Columns.User].isin(test_users)]
        ),
        prev_interactions=(
            base_models_data
            [[Columns.User, Columns.Item, Columns.Datetime, Columns.Weight]]
            [base_models_data[Columns.User].isin(test_users)]
        ),
        catalog=items['item_id'].unique()
    )

models_metrics: Dict[str, Dict[str, float]] = dict()
models_metrics['lfm'] = calc_metrics_(candidates, 'rank')
models_metrics['lfm']

{'Precision@10': 0.010847348226772332,
 'recall@10': 0.05638727084314944,
 'ndcg@10': 0.0128050773985203,
 'map@10': 0.023853619465689823,
 'novelty@10': 3.546669798241356}

In [29]:
# Вспоминаем про наши выборки интеракций для ранкера.
# Мы отобрали юзеров для обучения, валидации и теста.
# Оставляем среди них только тех, для кого есть и рекомы и таргеты

def users_filter(
    user_list: np.ndarray,
    candidates_df: pd.DataFrame,
    df: pd.DataFrame,
) -> pd.DataFrame:
    # Джойним интеракции на наших кандидатов для юзеров из трейна, вал и теста
    df = pd.merge(
        df[df['user_id'].isin(user_list)],
        candidates_df[candidates_df['user_id'].isin(user_list)],
        how='outer',  # right ?
        on=['user_id', 'item_id']
    )
    # Проставляем дефолтные значения интеракций
    min_score: float =  df['score'].min() - 0.01
    max_rank: int = df['rank'].max() + 1  # 101

    default_values = {
        'lfm_score': min_score, 'rank': max_rank,
        # Важно использовате те же дефолтные значения для интеракций,
        # чтобы не сделать утечку
        **interactions_default_values,
    }
    df.fillna(default_values, inplace=True)

    # Сортируем по user_id - это пригодится для вычисления рангов и групп для ранжирования
    df.sort_values(
        by=['user_id', 'item_id'],
        inplace=True,
    )
    return df

ranker_train = users_filter(train_users, candidates, base_models_data)
ranker_val = users_filter(val_users, candidates, base_models_data)
ranker_test = users_filter(test_users, candidates, base_models_data)

ranker_train.head()

/tmp/ipykernel_1189739/1450462766.py:27: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.fillna(default_values, inplace=True)
/tmp/ipykernel_1189739/1450462766.py:27: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.fillna(default_values, inplace=True)
/tmp/ipykernel_1189739/1450462766.py:27: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.is

,user_id,item_id,datetime,total_dur,weight,score,rank
2847010,21,24,2021-07-01,0.0,0.0,1.818225,99.0
2846970,21,101,2021-07-01,0.0,0.0,2.106124,58.0
2846931,21,142,2021-07-01,0.0,0.0,2.710972,14.0
2846994,21,366,2021-07-01,0.0,0.0,1.916106,83.0
2847000,21,416,2021-07-01,0.0,0.0,1.872532,89.0


In [18]:
models_metrics['listwise'] = calc_metrics_(ranker_test, 'listwise_rank')
models_metrics['listwise_hybrid'] = calc_metrics_(ranker_test, 'listwise_hybrid_rank')
pd.DataFrame(models_metrics)[['listwise', 'listwise_hybrid']]

/mnt/88fdd009-dda3-49d8-9888-cfd9d9d5910a/ITMO/RecomendationsService/.venv/lib/python3.10/site-packages/rectools/metrics/base.py:58: UserWarning: Expected integer dtype of 'rank' column in 'reco' dataframe.
  warnings.warn(f"Expected integer dtype of '{Columns.Rank}' column in 'reco' dataframe.")
/mnt/88fdd009-dda3-49d8-9888-cfd9d9d5910a/ITMO/RecomendationsService/.venv/lib/python3.10/site-packages/rectools/metrics/base.py:58: UserWarning: Expected integer dtype of 'rank' column in 'reco' dataframe.
  warnings.warn(f"Expected integer dtype of '{Columns.Rank}' column in 'reco' dataframe.")


,listwise,listwise_hybrid
Precision@10,0.010847,0.010847
recall@10,0.056387,0.056387
ndcg@10,0.012805,0.012805
map@10,0.023854,0.023854
novelty@10,3.546670,3.546670


In [19]:
models_metrics['listwise'] = calc_metrics_(ranker_test, 'listwise_rank')
pd.DataFrame(models_metrics)

/mnt/88fdd009-dda3-49d8-9888-cfd9d9d5910a/ITMO/RecomendationsService/.venv/lib/python3.10/site-packages/rectools/metrics/base.py:58: UserWarning: Expected integer dtype of 'rank' column in 'reco' dataframe.
  warnings.warn(f"Expected integer dtype of '{Columns.Rank}' column in 'reco' dataframe.")


,lfm,listwise,listwise_hybrid
Precision@10,0.010847,0.010847,0.010847
recall@10,0.056387,0.056387,0.056387
ndcg@10,0.012805,0.012805,0.012805
map@10,0.023854,0.023854,0.023854
novelty@10,3.546670,3.546670,3.546670


Теперь возмем модель второго уровня

In [20]:
def get_group(df: pd.DataFrame) -> np.ndarray:
    return np.array(
        df[['user_id', 'item_id']]
        .groupby(by=['user_id']).count()
        ['item_id']
    )

In [35]:
# Добавим таргет посложнее

def add_target(df: pd.DataFrame) -> pd.DataFrame:
    """
    0 - доля досмотра < 0.15
    1 - 0.15 <= доля досмотра < 0.75
    2 - 0.75 <= доля досмотра
    """
    df['target_ranker'] = (df[Columns.Weight] >= 15).astype(int)
    df['target_ranker'] += (df[Columns.Weight] >= 75).astype(int)
    return df

ranker_train = add_target(ranker_train)
ranker_val = add_target(ranker_val)
ranker_test = add_target(ranker_test)

ranker_train.head()
# ranker_train.iloc[[0,1,2,4,5]]

,user_id,item_id,datetime,total_dur,weight,score,rank,target_ranker
2847010,21,24,2021-07-01,0.0,0.0,1.818225,99.0,0
2846970,21,101,2021-07-01,0.0,0.0,2.106124,58.0,0
2846931,21,142,2021-07-01,0.0,0.0,2.710972,14.0,0
2846994,21,366,2021-07-01,0.0,0.0,1.916106,83.0,0
2847000,21,416,2021-07-01,0.0,0.0,1.872532,89.0,0


In [36]:
ranker_train.columns

Index(['user_id', 'item_id', 'datetime', 'total_dur', 'weight', 'score',
       'rank', 'target_ranker'],
      dtype='object')

In [ ]:
# Для обучения модели второго уровня используются только следующие фичи:
train_features = [
    'score', 'rank', 'target_ranker'
]

early_stopping_rounds = 32
params = {
    'objective': 'lambdarank',  # lambdarank, оптимизирующий ndcg
    'n_estimators': 10000,  # максимальное число деревьев
    'max_depth': 5,  # максимальная глубина дерева
    'num_leaves': 10,  # число листьев << 2^max_depth
    'min_child_samples': 100,  # число примеров в листе
    'learning_rate': 0.25,  # шаг обучения
    'reg_lambda': 1,  # L2 регуляризация
    'colsample_bytree': 0.9,  # доля колонок, которая используется в каждом дереве
    'early_stopping_rounds': early_stopping_rounds,  # число итераций, в течение которых нет улучшения метрик
    'verbose': early_stopping_rounds // 8,  # период вывода метрик
    'random_state': RANDOM_STATE,
}
fit_params = {
    'X': ranker_train[train_features],
    'y': ranker_train['target_ranker'],
    'group': get_group(ranker_train),
    'eval_set': [(ranker_val[train_features], ranker_val['target_ranker'])],
    'eval_group': [get_group(ranker_val)],
    'eval_metric': 'ndcg',
    'eval_at': (3, 5, 10),
    'feature_name': train_features,
}
second_model = LGBMRanker(**params)
second_model.fit(**fit_params)

[LightGBM] [Warning] early_stopping_round is set=32, early_stopping_rounds=32 will be ignored. Current value: early_stopping_round=32
[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.308813
[LightGBM] [Debug] init for col-wise cost 0.000022 seconds, init for row-wise cost 0.242445 seconds
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.085335 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Debug] Using Sparse Multi-Val Bin
[LightGBM] [Info] Total Bins 360
[LightGBM] [Info] Number of data points in the train set: 5824630, number of used features: 3
[LightGBM] [Warning] early_stopping_round is set=32, early_stopping_rounds=32 will be ignored. Current value: early_stopping_round=32
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 5 and depth = 4
Training until va

LGBMRanker(colsample_bytree=0.9, early_stopping_rounds=32, learning_rate=0.25,
           max_depth=4, min_child_samples=100, n_estimators=10000,
           num_leaves=10, objective='lambdarank', random_state=42, reg_lambda=1,
           verbose=4)

In [47]:
# Для обучения модели второго уровня используются только следующие фичи:
train_features = [
    'score', 'rank', 'target_ranker'
]

early_stopping_rounds = 32
params = {
    'objective': 'lambdarank',  # lambdarank, оптимизирующий ndcg
    'n_estimators': 10000,  # максимальное число деревьев
    'max_depth': 5,  # максимальная глубина дерева
    'num_leaves': 10,  # число листьев << 2^max_depth
    'min_child_samples': 100,  # число примеров в листе
    'learning_rate': 0.25,  # шаг обучения
    'reg_lambda': 1,  # L2 регуляризация
    'colsample_bytree': 0.9,  # доля колонок, которая используется в каждом дереве
    'early_stopping_rounds': early_stopping_rounds,  # число итераций, в течение которых нет улучшения метрик
    'verbose': early_stopping_rounds // 8,  # период вывода метрик
    'random_state': RANDOM_STATE,
}
fit_params = {
    'X': ranker_train[train_features],
    'y': ranker_train['target_ranker'],
    'group': get_group(ranker_train),
    'eval_set': [(ranker_val[train_features], ranker_val['target_ranker'])],
    'eval_group': [get_group(ranker_val)],
    'eval_metric': 'ndcg',
    'eval_at': (3, 5, 10),
    'feature_name': train_features,
}
second_model = LGBMRanker(**params)
second_model.fit(**fit_params)

with open('second_model.pkl', 'wb') as f:
    pickle.dump(second_model, f)

[LightGBM] [Warning] early_stopping_round is set=32, early_stopping_rounds=32 will be ignored. Current value: early_stopping_round=32
[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.308572
[LightGBM] [Debug] init for col-wise cost 0.000010 seconds, init for row-wise cost 0.158552 seconds
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.026249 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Debug] Using Sparse Multi-Val Bin
[LightGBM] [Info] Total Bins 360
[LightGBM] [Info] Number of data points in the train set: 5797004, number of used features: 3
[LightGBM] [Warning] early_stopping_round is set=32, early_stopping_rounds=32 will be ignored. Current value: early_stopping_round=32
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 7 and depth = 5
Training until va

LGBMRanker(colsample_bytree=0.9, early_stopping_rounds=32, learning_rate=0.25,
           max_depth=5, min_child_samples=100, n_estimators=10000,
           num_leaves=10, objective='lambdarank', random_state=1337,
           reg_lambda=1, verbose=4)

In [37]:
def add_score_and_rank(df: pd.DataFrame, y_pred_scores: np.ndarray, name: str) -> pd.DataFrame:
    # Добавляем скор модели второго уровня
    df[f'{name}_score'] = y_pred_scores
    # Добавляем ранг модели второго уровня
    df.sort_values(
        by=['user_id', f'{name}_score'],
        ascending=[True, False],
        inplace=True,
    )
    df[f'{name}_rank'] = df.groupby('user_id').cumcount() + 1

    return df

Получим ранжирование от модели второго уровня и посчитаем метрики

In [43]:
y_pred = second_model.predict(ranker_test[train_features])
ranker_test = add_score_and_rank(ranker_test, y_pred, 'target')
ranker_test.head()

,user_id,item_id,datetime,total_dur,weight,score,rank,target_ranker,target_score,target_rank
72130,3,3734,2021-07-22,5377.0,99.0,3.268239,6.0,2,0.499984,1
1098162,3,14,2021-07-01,0.0,0.0,1.832561,60.0,0,-0.499987,2
1098193,3,24,2021-07-01,0.0,0.0,1.597180,91.0,0,-0.499987,3
1098150,3,101,2021-07-01,0.0,0.0,1.949663,48.0,0,-0.499987,4
1098111,3,142,2021-07-01,0.0,0.0,2.940772,9.0,0,-0.499987,5


In [44]:
models_metrics['target'] = calc_metrics_(ranker_test, 'target')
pd.DataFrame(models_metrics)

/mnt/88fdd009-dda3-49d8-9888-cfd9d9d5910a/ITMO/RecomendationsService/.venv/lib/python3.10/site-packages/rectools/metrics/base.py:58: UserWarning: Expected integer dtype of 'rank' column in 'reco' dataframe.
  warnings.warn(f"Expected integer dtype of '{Columns.Rank}' column in 'reco' dataframe.")


,lfm,listwise,listwise_hybrid,target
Precision@10,0.010847,0.010847,0.010847,0.010847
recall@10,0.056387,0.056387,0.056387,0.056387
ndcg@10,0.012805,0.012805,0.012805,0.012805
map@10,0.023854,0.023854,0.023854,0.023854
novelty@10,3.546670,3.546670,3.546670,3.546670


Получим рекомендации для всех юзеров для прогона через сервис

In [ ]:
all_results = {}

for data_part in [ranker_train, ranker_val, ranker_test]:
    y_pred = second_model.predict(ranker_train[train_features])
    data = add_score_and_rank(ranker_train, y_pred, 'target')
    data.head()
    
    data_group = data.groupby('user_id').agg(list)
    data_group['item_id'] = data.apply(lambda row: [x for _, x in sorted(zip(row['target_rank'], row['item_id']))], axis=1)
    results_dict = data_group['item_id'].to_dict()
    all_results.update(results_dict)

with open('results_6.pkl', 'wb') as f:
    pickle.dump(all_results, f)